In [1]:
# Libraries
import pandas as pd
import numpy as np
import math
from math import erf, sqrt


----

### Question 3

The data set **''pca data''** consists of 392 observations each with 4 features, $y^{obs}_1 =
(12, 307, 130, 18)^t$ , . . . , $y_{392}^{obs} = (19.4, 119, 82, 31)^t$ $\in$ ${\rm I\!R^{4}}$, collected in the non-centred 4 × 392 data matrix $Y_{obs} = (y_1^{obs}, . . . , y_{392}^{obs})$.

In [2]:
pca_data = pd.read_csv('pca_data.csv', header = None)
pca_data.head()

,0,1,2,3,4,5,6,7,8,9,...,382,383,384,385,386,387,388,389,390,391
0,12,11.5,11,12,10.5,10,9,8.5,10,8.5,...,14.5,14.7,13.9,13,17.3,15.6,24.6,11.6,18.6,19.4
1,307,350.0,318,304,302.0,429,454,440.0,455,390.0,...,156.0,232.0,144.0,135,151.0,140.0,97.0,135.0,120.0,119.0
2,130,165.0,150,150,140.0,198,220,215.0,225,190.0,...,92.0,112.0,96.0,84,90.0,86.0,52.0,84.0,79.0,82.0
3,18,15.0,18,16,17.0,15,14,14.0,14,15.0,...,26.0,22.0,32.0,36,27.0,27.0,44.0,32.0,28.0,31.0


In [3]:
pca_data.shape

(4, 392)

##### Compute the covariance matrix **C** associated with the centred 4 × 392 data matrix


---

I will first calculate the centred the matrix $Y$ with bringing each of the dimensions to a 0 mean. 

$$\bar{\mathbf{y}} = \frac{1}{N} \Sigma^{N}_{n=1} y_n = 0$$

and then calculate the covariance matrix as follows:

$${C=\frac{1}{m}YY^t}$$

In [4]:
# first center the matrix 
# -----------------------
# calculate the mean of each feature
meanpoints = (pca_data.to_numpy()).mean(axis = 1)
print(meanpoints)

[ 15.54132653 194.4119898  104.46938776  23.44591837]


Reshape pca_dataframe for calculating with numpy broadcasting making it more elegant than looping through the dataframe.

First make it to numpy for broadcasting, then transpose to easily subtract the mean values array from each value in the numpy matrix. Then transpose again to regain the shape of $4x392$ matrix


In [5]:
# subtract feature mean from feature value in every given column
centred_matrix = (pca_data.to_numpy().transpose() - meanpoints).transpose()
print('The centred matrix is of shape: {}. {}'.format(centred_matrix.shape, '\n'))
                                                
# print the centred matrix means
print('And has the following means:{}{}'.format('\n', centred_matrix.mean(axis = 1).round()))

The centred matrix is of shape: (4, 392). 

And has the following means:
[ 0.  0. -0. -0.]


The means of the centred matrix show that each feature now has approx. zero mean.  (rounded values; minuscle)

---

Construct the 4x4 Covariance Matrix:

In [6]:
# set m to the number of observations
m = len(pca_data)

# calculate the cov matrix 
cov_mat = ((1/m)*(centred_matrix)).dot(centred_matrix.transpose())
cov_mat.shape

(4, 4)

In [7]:
print(cov_mat)

[[ 7.44007628e+02 -1.53462061e+04 -7.15402602e+03  8.91041531e+02]
 [-1.53462061e+04  1.07039843e+06  3.53271798e+05 -6.42789540e+04]
 [-7.15402602e+03  3.53271798e+05  1.44823408e+05 -2.28596122e+04]
 [ 8.91041531e+02 -6.42789540e+04 -2.28596122e+04  5.95474837e+03]]


The covariance matrix is a 4x4 matrix.


---

#### Compute the eigenvalues of **C**. What are the two largest eigenvalues $\lambda_1 and \lambda_2$?


Via Python Library:

In [8]:
# import functions for calculation of eigenvalues
from numpy import linalg as LA

#calculate eigenvalues and eigenvectors
eig_values, eig_vectors = LA.eig(cov_mat)
print('The eigenvalues are:{}{}{}'.format('\n',eig_values,'\n'))
print('The corresponding eigenvectors are:{}{}{}'.format('\n',eig_vectors,'\n'))

The eigenvalues are:
[1.19398056e+06 2.55829724e+04 3.53020170e+02 2.00404017e+03]

The corresponding eigenvectors are:
[[ 0.01412202 -0.07381036  0.99245729 -0.09685621]
 [-0.94565515 -0.32255402 -0.00656592  0.0406465 ]
 [-0.31976605  0.94260354  0.07987882  0.05355015]
 [ 0.05732875 -0.04482752  0.092762    0.99302524]]



These eigenvalues of my covariance matrix correspond to the variance in each of the D potential projection dimensions. Hence, I am choosing the two $\lambda 's$ with the highest value. Trying to capture most of the variance in the data through projecting onto these 2 dimensions.

In our example above it looks like the first 2 eigenvalues are the largest eigenvalues(capture most of the variance).

In [9]:
# Eigen value 1 and Eigenvalue 2
eig_values[:2].round(2)

array([1193980.56,   25582.97])

---


##### Construct the 2 × 392-matrix **X** corresponding to the projection of the centred data matrix **Y** onto the two directions $e_1 and e_2$ of largest variance. 

###### In your result you only need to indicate the structure of **X**, e.g., by specifing the entries $X_{1,1} = \langle y_1, e_1 \rangle$ , $X_{1,392} = \langle y_{392}, e_1 \rangle$, $X_{2,1} = \langle y_1, e_2 \rangle$ and $X_{2,392} = \langle y_{392}, e_2 \rangle$

----

The approach taken will be to construct the 2x392 matrix as follows: 
$$ X = E^tY $$

where **E** is my new $k$x$d$ matrix:   $$E=(e_1,e_2,...e_d)$$
with the d columns being the eigenvectors of the d largest eigenvalues of interest.

In [10]:
# first construct my E
E = eig_vectors[:,:2]
E

array([[ 0.01412202, -0.07381036],
       [-0.94565515, -0.32255402],
       [-0.31976605,  0.94260354],
       [ 0.05732875, -0.04482752]])

In [11]:
# show shape of E
E.shape

(4, 2)

In [12]:
# transpose E
E_t = E.transpose()

# show shape of transposed E
E_t.shape

(2, 4)

In [13]:
# Show shape of my centred matrix
centred_matrix.shape

(4, 392)

In [14]:
# Construct the 2 × 392-matrix X
projected_matrix = E_t.dot(centred_matrix)
projected_matrix.shape

(2, 392)

In [15]:
projected_M_df = pd. DataFrame(data=projected_matrix, index=None, columns=None)
projected_M_df[[*range(12)]]

,0,1,2,3,4,5,6,7,8,9,10,11
0,-114.995473,-167.029503,-131.807122,-118.668486,-113.543369,-252.309722,-283.057405,-268.226463,-285.587768,-212.892226,-199.856136,-156.080877
1,-11.744956,7.547733,3.632831,8.164432,-0.550607,13.282598,26.074663,25.914307,30.391316,18.432092,1.727184,6.363419


Above are the first 12 values of the calculated X. 

------

Specifiying the entries in the following way: 
$$X_{1,1} = \langle y_1, e_1 \rangle$$ 

$$X_{1,392} = \langle y_{392}, e_1 \rangle$$

$$X_{2,1} = \langle y_1, e_2 \rangle$$

$$X_{2,392} = \langle y_{392}, e_2 \rangle$$
is done below: 

In [16]:
# extract e1
e1 = E_t[0].reshape(-1,1)
print(e1)

[[ 0.01412202]
 [-0.94565515]
 [-0.31976605]
 [ 0.05732875]]


In [17]:
# extract e2
e2 = E_t[1].reshape(-1,1)
print(e2)

[[-0.07381036]
 [-0.32255402]
 [ 0.94260354]
 [-0.04482752]]


In [18]:
# turn centred_matrix numpy to pandas dataframe for easier extraction
centred_as_df = pd. DataFrame(data=centred_matrix, index=None, columns=None)
centred_as_df

,0,1,2,3,4,5,6,7,8,9,...,382,383,384,385,386,387,388,389,390,391
0,-3.541327,-4.041327,-4.541327,-3.541327,-5.041327,-5.541327,-6.541327,-7.041327,-5.541327,-7.041327,...,-1.041327,-0.841327,-1.641327,-2.541327,1.758673,0.058673,9.058673,-3.941327,3.058673,3.858673
1,112.588010,155.588010,123.588010,109.588010,107.588010,234.588010,259.588010,245.588010,260.588010,195.588010,...,-38.411990,37.588010,-50.411990,-59.411990,-43.411990,-54.411990,-97.411990,-59.411990,-74.411990,-75.411990
2,25.530612,60.530612,45.530612,45.530612,35.530612,93.530612,115.530612,110.530612,120.530612,85.530612,...,-12.469388,7.530612,-8.469388,-20.469388,-14.469388,-18.469388,-52.469388,-20.469388,-25.469388,-22.469388
3,-5.445918,-8.445918,-5.445918,-7.445918,-6.445918,-8.445918,-9.445918,-9.445918,-9.445918,-8.445918,...,2.554082,-1.445918,8.554082,12.554082,3.554082,3.554082,20.554082,8.554082,4.554082,7.554082


In [19]:
# extract y1
y1 = centred_as_df[0].to_numpy()
print(y1)

[ -3.54132653 112.5880102   25.53061224  -5.44591837]


In [20]:
# extract y392
y392 = centred_as_df[391].to_numpy()
print(y392)

[  3.85867347 -75.4119898  -22.46938776   7.55408163]


That means: 

$$X_{1,1} = \Bigg \langle (-3.54, 112.59, 25.53, -5.45), 
\begin{pmatrix} 0.01 \\ 0.99 \\ -0.95 \\ -0.01 \end{pmatrix} \Bigg \rangle$$ 

$$X_{1,392} = \Bigg \langle  (3.86, -75.41, -22.47, 7.55), 
\begin{pmatrix} 0.01 \\ 0.99 \\ -0.95 \\ -0.01 \end{pmatrix} \Bigg \rangle$$ 

$$X_{2,1} =\Bigg \langle (-3.54, 112.59, 25.53, -5.45),
\begin{pmatrix} -0.07 \\ -0.1 \\ -0.32 \\ 0.04 \end{pmatrix} \Bigg \rangle$$ 


$$X_{2,392} =\Bigg \langle (3.86, -75.41, -22.47, 7.55), 
\begin{pmatrix} -0.07 \\ -0.1 \\ -0.32 \\ 0.04 \end{pmatrix} \Bigg \rangle$$ 


_Note: Values are rounded for nicer visualisation._


----

Below individual calculations of $X_{1,1}$, $X_{1,392}$, $X_{2,1}$ and $X_{2,392}$ for comparison with table above.

In [21]:
# Calculate X1,1
x11 = y1.dot(e1)
print(x11[0].round(8))
print(projected_M_df[0][0].round(8))

# Compare: equal to calculated value in dataframe above?
print(x11[0].round(8) == projected_M_df[0][0].round(8))

-114.99547273
-114.99547273
True


$X_{1,1} = -114.99547273$

In [22]:
# Calculate X1,392
x1392 = y392.dot(e1)
print(x1392[0].round(8))
print(projected_M_df[391][0].round(8))

# Equal to calculated value in dataframe above?
print(x1392[0].round(8) == projected_M_df[391][0].round(8))

78.98624197
78.98624197
True


$X_{1,392} = 78.98624197$

In [23]:
# Calculate X2,1
x21 = y1.dot(e2)
print(x21[0].round(8))
print(projected_M_df[0][1].round(8))

# Equal to calculated value in dataframe above?
print(x21[0].round(8) == projected_M_df[0][1].round(8))

-11.74495591
-11.74495591
True


$X_{2,1} = -11.74495591$

In [24]:
# Calculate X2,392
x2392 = y392.dot(e2)
print(x2392[0].round(4))
print(projected_M_df[391][1].round(4))

# Equal to calculated value in dataframe above?
print(x2392[0].round(8) == projected_M_df[391][1].round(8))

2.5213
2.5213
True


$X_{2,392} = 2.5213$